In [1]:
import os
from chainbase_sdk.sql_alpha import ChainbaseSQLAlpha

CHAINBASE_API_KEY=os.environ['CHAINBASE_API_KEY']

L2_CHAIN = 'optimism'
L1_PORTAL = '0x25ace71c97B33Cc4729CF772ae268934F7ab5fA1'
#L1_PORTAL = '0x49048044d57e1c92a77f79988d21fa8faf74e97e'

In [2]:
client = ChainbaseSQLAlpha(api_key=CHAINBASE_API_KEY)

In [3]:
def withdrawal_basic_query(client, chain, contract_address):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT * FROM aggregate_data
    """
    # missing and success = true

    df_l1_finalized = client.query_pandas(query)

    return df_l1_finalized

In [4]:
def deposit_basic_query(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS BIGINT) AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
    """
    df_deposit = client.query_pandas(query)

    return df_deposit

In [ ]:
base_withdrawals = withdrawal_basic_query(client=client, chain=L2_CHAIN,contract_address=L1_PORTAL)

len(base_withdrawals)

In [ ]:
base_withdrawals.head()

In [ ]:
base_deposits = deposit_basic_query(client=client, contract_address=L1_PORTAL)

len(base_deposits)

In [ ]:
base_deposits.head()

In [ ]:
# save to csv

base_withdrawals.to_csv(f'../../data/{L2_CHAIN}/basic_withdrawals.csv', index=False)
base_deposits.to_csv(f'../../data/{L2_CHAIN}/basic_deposits.csv', index=False)

## tables for the frontend
as the api limits to 10_000 rows, it is necessary for me to manipulate the data directly within sql. hence i have made three queries to extract all views i am interested in for each chain.

In [10]:
def counters(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE)/1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT
    SUM(value) AS total_value,
    SUM(IF(finalized_transaction_hash IS NULL, value, 0)) AS sum_if_finalized_null,
    SUM(IF(proven_withdrawal_hash IS NULL, value, 0)) AS sum_if_proven_null,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 and proven_withdrawal_hash is null, value, 0)) AS not_proven_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 and proven_withdrawal_hash is null, value, 0)) AS not_proven_4w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_4w    
FROM 
    aggregate_data
"""
    df = client.query_pandas(query)
    return df

In [ ]:
df_counters = counters(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

In [ ]:
df_counters

In [13]:
df_counters.to_csv(f'../../data/{L2_CHAIN}/counters.csv', index=False)

In [14]:
def daily_volumes(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
deposits_basic AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS DOUBLE) / 1e18 AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
),
aggregated_data AS (
    SELECT 
        DATE(iw.block_timestamp) AS date,
        SUM(iw.value) AS initiated_volume,
        COALESCE(SUM(pw.initiated_value), 0) AS proved_volume,
        COALESCE(SUM(fw.initiated_value), 0) AS finalized_volume,
        COALESCE(SUM(dep.value), 0) AS deposited_volume
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    LEFT JOIN 
        deposits_basic dep ON DATE(iw.block_timestamp) = DATE(dep.block_timestamp)
    GROUP BY 
        DATE(iw.block_timestamp)
),
cumulative_data AS (
    SELECT 
        date,
        initiated_volume,
        proved_volume,
        finalized_volume,
        deposited_volume,
        (COALESCE(deposited_volume, 0) - COALESCE(initiated_volume, 0)) AS daily_difference,
        SUM(initiated_volume) OVER (ORDER BY date) AS cumulative_initiated,
        SUM(proved_volume) OVER (ORDER BY date) AS cumulative_proved,
        SUM(finalized_volume) OVER (ORDER BY date) AS cumulative_finalized,
        SUM(deposited_volume) OVER (ORDER BY date) AS cumulative_deposited
    FROM 
        aggregated_data
)
SELECT 
    date,
    initiated_volume,
    proved_volume,
    finalized_volume,
    deposited_volume,
    daily_difference,
    cumulative_initiated,
    cumulative_proved,
    cumulative_finalized,
    cumulative_deposited
FROM 
    cumulative_data
ORDER BY 
    date
 
"""
    df = client.query_pandas(query)
    return df

In [ ]:
df_daily = daily_volumes(chain=L2_CHAIN, l1_portal_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

In [ ]:
print(df_daily)
df_daily.to_csv(f'../../data/{L2_CHAIN}/daily_volumes.csv', index=False)

In [17]:
def stuck_in_contract(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
withdrawals_not_withdrawn AS (
    SELECT 
        iw.withdrawal_hash,
        iw.value AS initiated_value,
        iw.block_timestamp
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    WHERE 
        pw.withdrawal_hash IS NULL 
        AND fw.withdrawal_hash IS NULL
        AND iw.block_timestamp < DATE_SUB(CURRENT_DATE, INTERVAL 14 DAY)
),
daily_not_withdrawn AS (
    SELECT 
        DATE(block_timestamp) AS date,
        SUM(initiated_value) AS total_not_withdrawn
    FROM 
        withdrawals_not_withdrawn
    GROUP BY 
        DATE(block_timestamp)
)
SELECT 
    date,
    COALESCE(total_not_withdrawn, 0) AS total_not_withdrawn
FROM 
    daily_not_withdrawn
ORDER BY 
    date
"""

    df = client.query_pandas(query)
    return df

In [ ]:
df_stuck = stuck_in_contract(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

In [19]:
df_stuck.to_csv(f'../../data/{L2_CHAIN}/forgotten_funds.csv', index=False)